In [44]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print('done importing')

done importing


In [59]:
def rate(directory,rate):
    # estimating the most dangerous non must stations based on the added risk
    
    added_risk = pd.read_csv(f'{directory}\\added_risk_by_station_smooth.csv')
    added_risk.index = added_risk['Unnamed: 0']
    added_risk = added_risk.drop(columns='Unnamed: 0')
    
    #%%
    
    # lets read all the meters so we can map from meter number to the name of the station
    
    meters = pd.read_csv('..\data\\OPC-Meters.csv')
    def meter_to_name(meterId):
        name = meters.loc[meters['meterId']==meterId]['name']
        if len(name) ==0:
            return meterId
        return name.values[0]
    
    #%%
    
    # and create the reverse mapper needed for renaming the columns
    new_columns = {column: meter_to_name(column) for column in added_risk.columns}
    
    # and rename them
    added_risk = added_risk.rename(columns=new_columns)
    
    
    #%%
    
    # reading the must stations (actualy customers)
    
    must_customers = pd.read_csv("..\data\\must_meters.csv")
    must_customers_vals = must_customers['Must Customers'].values
    
    #%%
    
    # and deriving the actual stations we need to keep (one customer -> many stations/meters)
    must_station = [station for station in added_risk.columns for must in must_customers_vals if must in station]
    pd.Series(must_station).to_csv('..\data\\must_stations.csv')
    
    # finally lets drop them as they are not important in the risk analysis, they will stay anyway
    added_risk_non_must = added_risk.drop(columns = must_station)
    added_risk_non_must.to_csv(f'{directory}\\added_risk_non_must.csv')

    #%%
    
    # argsort gives us the argument, which we want to convert to the meter name
    mapper = {idx : column for idx,column in enumerate(added_risk_non_must.columns)}
    def f(idx):
        return mapper[idx]
    
    # sorting
    added_risk_sorted_idx = added_risk_non_must.values.argsort(axis=1)
    # mapping argument index to name and arrange in descending order
    stations_by_risk = pd.DataFrame(added_risk_sorted_idx).applymap(f).loc[::,::-1]
    # re indexing
    stations_by_risk.index = added_risk_non_must.index
    stations_by_risk.to_csv(f'{directory}\\stations_by_risk.csv')
    
    #%%
    
    # finally lets save the meters in the order of risk so we can iterate over them and add one by one
    max_risk_stations = stations_by_risk.iloc[-1]
    max_risk_stations.to_csv(f'{directory}\\max_risk_stations.csv',index=False)
    
    
    #%%
    
    # now we need to rename all the aggregates files so we can iterate over them 
    import os
    for file in os.listdir(f"{directory}\stations_agg"):
        name = (file.split('.')[0])
        new_name = meter_to_name(name)
        if f'{new_name}.csv' not in os.listdir(f"{directory}\stations_agg"):
            os.rename(f'{directory}\stations_agg\\{file}',f'{directory}\stations_agg\\{new_name}.csv')
    
    #%%
    
    # next thing we need to add these one by one until we reach a certain capacity
    # first lets add all the must statiosn
    
    must_pt = None
    maxes = {}
    
    #%%
    
    for idx,station in enumerate(must_station):
        if must_pt is None :
            print('first time')
            must_pt =  pd.read_csv(f'{directory}\\stations_agg\\{station}.csv',index_col='timeFrom').fillna(0)
        else:
            station_pt = pd.read_csv(f'{directory}\\stations_agg\\{station}.csv',index_col='timeFrom').fillna(0)
            must_pt = must_pt.add(station_pt,fill_value=0)
        print(f'done adding station {station}, {idx}/{len(must_station)}')
    
    #%%
    
    # adjusting so it would be kWh
    must_pt *= 2
    
    #%%
    
    must_pt.max().max()
    
    #%%
    
    least_risky = max_risk_stations[::-1]
    all_pt = must_pt
    non_must_pt = None
    for idx,station in enumerate(least_risky):
        station = station.split('.')[0]
        
        station_pt = pd.read_csv(f'{directory}\stations_agg\\{station}.csv',index_col='timeFrom').fillna(0)
        # again adjusting the consumption rate
        station_pt *= 2
        all_pt = all_pt.add(station_pt,fill_value=0)
        if all_pt.max().max() > rate*1000:
            all_pt = all_pt.subtract(station_pt,fill_value=0)
            break
        print(f'done adding station {station}, {idx}/{len(least_risky)}')
    
    #%%
    
    all_pt.max().max()
    least_risky[:225].to_csv(f'{directory}\\stations_to_{rate}.csv')
    

    #%%
    
    all_pt.to_csv(f'{directory}\\aggregate_to_{rate}.csv',header=True)   

In [ ]:
rate('..\data\\April-2019-based',400)
rate('..\data\\April-2020-based',400)
rate('..\data\\May-2020-based',400)

In [60]:
rate('..\data\\April-2019-based',400)
rate('..\data\\April-2020-based',400)
rate('..\data\\May-2020-based',400)

first time
done adding station Perrigo, 0/22
done adding station Galam, 1/22
done adding station Starplast, 2/22
done adding station Perrigo XPW, 3/22
done adding station Tambour Ascar, 4/22
done adding station Tambour Akko South, 5/22
done adding station Tambour Tzah Sharpon, 6/22
done adding station Hadera Paper Carmel, 7/22
done adding station Hadera Paper Frenkel, 8/22
done adding station Hadera Paper Matash, 9/22
done adding station Hadera Paper Total PM, 10/22
done adding station Intel IDCj, 11/22
done adding station Iscar MicroTools, 12/22
done adding station Iscar ETM, 13/22
done adding station Iscar MicroDent, 14/22
done adding station Iscar Plot 130, 15/22
done adding station Gadot Main, 16/22
done adding station Intel F28, 17/22
done adding station Intel IDC1-4, 18/22
done adding station Intel IDC9, 19/22
done adding station Intel IDPj, 20/22
done adding station Iscar MainTools, 21/22
done adding station PGW, 0/276
done adding station Glil Yam 2, 1/276
done adding station Po